# **Project 2: AI Chat For Banking Transactions**

**Group Member:**
1. P. Balaji  (23070243064)
2. Praveen Choudhary (23070243065)
3. Soumya Ranjan Nayak (23070243063)
4. Mansi Kanojiya (23070243026)
5. Shubham Durgude (23070243014)


In [1]:
# Install Library
# !pip install psycopg2
# !pip install langchain langchain-google-genai
# !pip install --upgrade jupyter ipywidgets
# !pip install PyPDF2
# !pip install gradio

In [150]:
# Import Some Library
import psycopg2 as ps
import datetime
import pandas as pd
import os
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
import PyPDF2
import gradio as gr

# **Auto Insert And Question Answer System**

In [ ]:
class DatabaseHandler:
    def __init__(self, db_name, user, password, port):
        self.db_name = db_name
        self.user = user
        self.password = password
        self.port = port
        self.connection = None
        self.cursor = None
    
    def connect(self):
        """Connect to the PostgreSQL database."""
        try:
            self.connection = ps.connect(database=self.db_name, user=self.user, password=self.password, port=self.port)
            self.cursor = self.connection.cursor()
            print("Connected to the database")
        except Exception as e:
            print(f"Error connecting to database: {e}")
    
    def execute_query(self, query):
        """Execute the provided SQL query."""
        try:
            self.cursor.execute(query)
            self.connection.commit()
            print("*****************************************")
            print(query)
            print("*****************************************")
            print("Query executed successfully")
            print("*****************************************")
        except Exception as e:
            print(f"Error executing query: {e}")
    
    def fetch_results(self):
        """Fetch results of the last executed query."""
        return self.cursor.fetchall()

    def close(self):
        """Close the database connection."""
        if self.connection:
            self.connection.close()
            print("Connection closed.")

class SQLGenerator:
    def __init__(self, model, temperature):
        self.model = model
        self.temperature = temperature
        os.environ["GOOGLE_API_KEY"] = "YOUR API KEY" 

    def create_chain(self, template):
        """Create a LangChain LLMChain for generating SQL queries."""
        prompt = PromptTemplate(input_variables=["question"], template=template)
        llm = ChatGoogleGenerativeAI(model=self.model, temperature=self.temperature)
        chain = LLMChain(llm=llm, prompt=prompt)
        return chain

    def clean_sql_query(self, query):
        """Clean the SQL query to remove unwanted characters or formatting."""
        query = query.strip()
        
        query = query.replace("```sql", "").replace("```", "")
        
        return query

    def generate_sql_from_question(self, question, template):
        """Generate SQL query based on a natural language question."""
        chain = self.create_chain(template)
        sql_query = chain.run({"question": question})
        return self.clean_sql_query(sql_query)

    def generate_sql_from_pdf_text(self, pdf_text, template):
        """Generate SQL insert query based on extracted PDF text."""
        chain = self.create_chain(template)
        sql_query = chain.run({"question": pdf_text})
        return self.clean_sql_query(sql_query)


class PDFExtractor:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path

    def extract_text(self):
        """Extract text from the provided PDF file."""
        pdf_name = os.path.basename(pdf_path)

        full_text = "file name: " + pdf_name 
        try:
            with open(self.pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page_num in range(len(reader.pages)):
                    page = reader.pages[page_num]
                    full_text += page.extract_text()
        except Exception as e:
            print(f"Error extracting PDF text: {e}")
        return full_text

class BankSystem:
    def __init__(self, db_config, model="gemini-pro", temperature=0.3):
        self.db_handler = DatabaseHandler(**db_config)
        self.sql_generator = SQLGenerator(model, temperature)
        self.result_formatter = LLMChain(
            prompt=PromptTemplate(
                input_variables=["question", "result"],
                template="""
                You are a helpful assistant that translates all generate answer from the result into proper wording.
                if data resule in single measure like avg and sum then just write in proper wording formate not in tabluar formate.  
                If all data result is tabular then just print this in a proper table format with column (transaction_id, file_path, description, transaction_date, credit, debit, balance). 
                Also, give the answer according to the question asked by the user.
                If the question is out of the box, then just say this data is not available.

                Question: {question}
                Result: {result}
                Proper Wording:
                """
            ),
            llm=ChatGoogleGenerativeAI(model=model, temperature=temperature)
        )
        
    def connect_to_db(self):
        """Connect to the database."""
        self.db_handler.connect()

    def insert_data_from_pdf(self, pdf_path):
        """Insert data into the database from PDF."""
        pdf_extractor = PDFExtractor(pdf_path)
        pdf_text = pdf_extractor.extract_text()
        
        insert_template = """
        You are a helpful assistant that translates pdf text data into SQL queries for generating the query 
        to insert the data into the table transactions. The table contains columns:
        file path, description, transaction_date, credit, debit, balance. 
        Always return the direct SQL query as the answer. Do not include anything else.
        
        PDF text data: {question}
        SQL Query:
        """
        
        sql_query_insert = self.sql_generator.generate_sql_from_pdf_text(pdf_text, insert_template)
        self.db_handler.execute_query(sql_query_insert)
    
    def answer_question(self, question):
        """Answer user’s question by generating SQL query and formatting the results."""
        question_template = """
        You are a helpful assistant that translates natural language questions into SQL queries and make sure also give source to all the query for all the user question.
        Generate a valid SQL query based on the given question. In question if user ask about the Source it mean it talk about the file_path. (source = file_path)
        Use the table details: transactions (file_path, description, transaction_date, credit, debit, balance).
        If the question is out of scope, respond with 'this data is not available'. 
        Example: Question: total sum of invest on Food by praveen? 
                 Answer: SELECT  SUM(debit) AS total_investment_on_food FROM transactions WHERE description LIKE '%Food%' AND file_path like '%Praveen%';

        Question: {question}
        SQL Query:
        """
        
        sql_query = self.sql_generator.generate_sql_from_question(question, question_template)
        self.db_handler.execute_query(sql_query)
        result = self.db_handler.fetch_results()

        # Format results into proper wording or table
        formatted_result = self.result_formatter.invoke({"question": question, "result": result})
        return formatted_result

    def close_db_connection(self):
        """Close the database connection."""
        self.db_handler.close()

# Example Usage:

# Database connection details
db_config = {
    "db_name": "bank",
    "user": "postgres",
    "password": "Helping@1034",
    "port": 5433
}

# Initialize the bank system
bank_system = BankSystem(db_config)

# Connect to the database
bank_system.connect_to_db()

Connected to the database


## **PDF Data Inserted Into The Database PG-Admin**

In [67]:
# Insert data from PDF
#pdf_path = "C:\\Users\\praveen choudhary\\Desktop\\Praveen Statement.pdf" 
#pdf_path = "C:\\Users\\praveen choudhary\\Desktop\\Balaji Statement.pdf"
#pdf_path = "C:\\Users\\praveen choudhary\\Desktop\\Mansi Statement.pdf"
#pdf_path = "C:\\Users\\praveen choudhary\\Desktop\\Shubham Statement.pdf"
pdf_path = "C:\\Users\\praveen choudhary\\Desktop\\Soumya Statement.pdf"

bank_system.insert_data_from_pdf(pdf_path)

*****************************************

INSERT INTO transactions (file_path, description, transaction_date, credit, debit, balance) VALUES
('Soumya Statement.pdf', 'Payment - Zomato', '01/05/2024', 500.00, 1500.00, 174800.00),
('Soumya Statement.pdf', 'Payment - Zomato', '02/05/2024', 100.00, 10000.00, 164900.00),
('Soumya Statement.pdf', 'Payment - Deposit', '05/05/2024', 5000.00, 10.00, 169890.00),
('Soumya Statement.pdf', 'Payment - Deposit', '06/05/2024', 10000.00, 500.00, 179390.00),
('Soumya Statement.pdf', 'Payment - Electricity', '08/05/2024', 0.00, 1000.00, 178390.00),
('Soumya Statement.pdf', 'Payment - Swigge', '09/05/2024', 0.00, 5000.00, 173390.00),
('Soumya Statement.pdf', 'Payment - Amazon', '10/05/2024', 0.00, 500.00, 172890.00),
('Soumya Statement.pdf', 'Payment - Account Transfer Out', '15/05/2024', 0.00, 10000.00, 162890.00);

*****************************************
Query executed successfully
*****************************************


## **Question Answer**

In [152]:
# Answer a question
# user_question = input("Ask your question: ")
user_question = "What is the total amount invested in 'Food' by Praveen?"

result = bank_system.answer_question(user_question)
print(result['text'])
# print(result)

# Close the database connection
# bank_system.close_db_connection()

*****************************************

SELECT  SUM(debit) AS total_investment_on_food
FROM transactions
WHERE description LIKE '%Food%'
AND file_path like '%Praveen%';

*****************************************
Query executed successfully
*****************************************
Praveen has invested a total of 9000 in 'Food'.


In [146]:
print(result['text'])

Praveen's total investment in Food is 9000.


# **Interface**

In [148]:
import gradio as gr

# Define Gradio functions for PDF insertion and question answering
def insert_pdf_to_db(pdf_file):
    if pdf_file is None:
        return "Please upload a PDF file."
    
    try:
        # Insert data from the uploaded PDF into the database
        pdf_path = pdf_file.name
        bank_system.insert_data_from_pdf(pdf_path)
        return "PDF data inserted successfully into the database."
    except Exception as e:
        return f"Error processing the PDF: {e}"

def ask_question(question):
    try:
        result = bank_system.answer_question(question)
        return result['text']  # Assuming `result` has a `text` key for formatted output
    except Exception as e:
        return f"Error answering the question: {e}"

# Define Gradio interface components
with gr.Blocks() as bank_interface:
    with gr.Tab("Upload PDF"):
        gr.Markdown("### Upload your bank statement PDF")
        pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
        pdf_output = gr.Textbox(label="Status", interactive=False)
        pdf_submit = gr.Button("Insert into Database")
        pdf_submit.click(insert_pdf_to_db, inputs=pdf_input, outputs=pdf_output)

    with gr.Tab("Question Answering"):
        gr.Markdown("### Ask a question about the transactions data")
        question_input = gr.Textbox(label="Your Question", placeholder="Type your question here...", lines=3)
        answer_output = gr.Textbox(label="Answer", lines=10, interactive=False)
        question_submit = gr.Button("Get Answer")
        question_clear = gr.Button("Clear Question")

        question_submit.click(ask_question, inputs=question_input, outputs=answer_output)
        question_clear.click(lambda: "", inputs=None, outputs=[question_input, answer_output])

# Launch the Gradio app
bank_interface.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


*****************************************

INSERT INTO transactions (
  file_path,
  description,
  transaction_date,
  credit,
  debit,
  balance
)
VALUES
  ('Soumya Statement.pdf', 'Payment - Credit Card', '12/01/2023', 5400.00, NULL, 170400.00),
  ('Soumya Statement.pdf', 'Payment - Insurance', '13/02/2023', 3000.00, NULL, 167400.00),
  ('Soumya Statement.pdf', 'Account Transfer In', '16/02/2023', 500000.00, NULL, 667400.00),
  ('Soumya Statement.pdf', 'Cheque Deposit', '20/02/2023', 10000.00, NULL, 677400.00),
  ('Soumya Statement.pdf', 'Payment - Electricity', '24/03/2023', NULL, 1500.00, 675900.00),
  ('Soumya Statement.pdf', 'Payment - Water Utility', '25/03/2023', NULL, 600.00, 675300.00),
  ('Soumya Statement.pdf', 'Payment - Car Loan', '02/04/2023', NULL, 3500.00, 671800.00),
  ('Soumya Statement.pdf', 'Account Transfer Out', '05/05/2023', NULL, 80000.00, 591800.00);

*****************************************
Query executed successfully
***************************************

In [ ]:
# Q1. "how many transaction happen on Amazon also print source"
# Q2. "how many transaction happen on Zomato by Shubham"
# Q3. "how many invest happen on Zomato by Praveen"